# **WATER QUALITY EXPLANATORY DATA ANALYSIS**
---


## **What to expect from this project?**

*   Data Preprocessing (Profiling, Cleaning, Reduction and Transformation)
        [Profiling, Cleaning, Reduction/Filtering, Transformation]
*   Correlation Analysis (Bivariate and multivariate)
        [Check on the relationship of all dimensions]
*   Geospatial (Regional) Analysis
        [Based on water quality and population; List of regional priorities on improving water potability]

## **Context**

*   Access to safe drinking-water is essential to health, a basic human right and a component of effective policy for health protection. This is important as a health and development issue at a national, regional and local level. In some regions, it has been shown that investments in water supply and sanitation can yield a net economic benefit, since the reductions in adverse health effects and health care costs outweigh the costs of undertaking the interventions.
*   Drinking water and staying hydrated is associated with a reduced incidence of urinary tract infections (UTIs), lower blood pressure and heart disease. Therefore, drinking water is essential for good heart health.
*   Water is the most important nutrient for the body. It has many benefits for your health and helps to protect you from illness and disease. Water is also an essential part of a healthy lifestyle.

## **Table of Contents**

1.   Introduction
2.   Preparation
3.   Data Preprocessing
4.   Regional Analysis
5.   [Conclusion](https://colab.research.google.com/drive/18h88Xyflx059_vvjcD2gola6W_p_OqZW#scrollTo=4y-VEateho9o&line=1&uniqifier=1)


# **Preparation**
---



## **Python Libraries**

In [ ]:
## This notebook is run in Python 3 with a handful of pre-installed analytics libraries 
## There

import pandas as pd # dataframe processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # linear algebra
import matplotlib.pyplot as plt # data visualisation
import seaborn as sns # data visualisation
import plotly.express as px # data visualisation
import missingno as msno # identify and visualise missing data

# **Exploration**
---




### First Insight

### Second Insight

# **Geospatial Analysis**
---




### First Insight

### Second Insight

# **Conclusion**
---




1.   Idea 1
2.   Idea 2

# **NOTES**
---



### **Cool Usable Codes**



*   ***Seaborn Pair Plot with Hue*** (Multivariate Analysis; Multi plot charts)
        sns.pairplot (data = dataframe, hue = 'column1')
*   ***Missingno Matrix Plot***(Show missing data in the dataframe)
        When data is present, the plot is shaded in grey, and when it is absent the plot is displayed in white.
        msno.matrix (dataframe)
*   List item
        Insert code


### Links To Go Back Later

*   [Distribution and characteristics of wastewater treatment plants within the global river network](https://essd.copernicus.org/articles/14/559/2022/)
*   [Using the missingno Python library to Identify and Visualise Missing Data Prior to Machine Learning](https://towardsdatascience.com/using-the-missingno-python-library-to-identify-and-visualise-missing-data-prior-to-machine-learning-34c8c5b5f009)
*   [Python Jupyter - Insight to Univariate and Multivariate Distribution Plots](https://www.youtube.com/watch?v=vssXHSLSGL4&ab_channel=NomanArshed)
*   [Univariate, Bivariate, and Multivariate Data Analysis in Python](https://medium.com/mlearning-ai/univariate-bivariate-and-multivariate-data-analysis-in-python-341493c3d173#:~:text=Multivariate%20analysis%20is%20a%20more,plot%2C%20with%20Hue%20as%20Approved.)
*   List item




